In [1]:
from os import listdir
from os.path import isfile, join
import numpy as np
import re
from md.post import disptools
from scipy.interpolate import CubicSpline, interp1d

# change from metal to Si units
metal2si = 1.60217662e-19*1e20*6.0221409e23*1000

# Get the files from directory
mypath = "../workspace/"
fpattern = re.compile('ArPrim.*.log')
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
onlylogfiles = [f for f in onlyfiles if fpattern.match(f)]
regex = '[+-]?[0-9]'
onlylogfiles.sort(key=lambda f: int(''.join(re.findall(regex, f))))
nfiles = len(onlylogfiles)

eig_full = []
q_full = []
# Get dispersion data

for ifile in range(nfiles):
    eig, q = disptools.get_data(join(mypath, onlylogfiles[ifile]))
    eig = np.sqrt(eig*metal2si)/1e12/2/np.pi # change to THz
    eig_full.append(eig)
    q_full.append(q)

neigvals = eig_full[0].shape[1]

# Special symmetry points in FCC
S = np.array([0,0,0], dtype=float)
X = np.array([0.5,0,0.5], dtype=float)
W = np.array([0.5,0.25,0.75], dtype=float)
L = np.array([0.5,0.5,0.5], dtype=float)
Xp = np.array([0.5,1.0,0.5], dtype=float)
K = np.array([3./4.,3./8.,3./8.], dtype=float)
U = np.array([5/8,1/4,5/8], dtype=float)
W = np.array([3/4,1/4,1/2], dtype=float)    

# Create plot data
# qx qy qz qr w1 w2 w3 ....
SPs = [S,X,W,K,S,L,U,W,L]
SPsNames = ['Γ','X','W','K','Γ','L','U','W','L']
disp_data_full = []
disp_inter_full = []
nintp      = 10

for ifiles in range(nfiles):
    qr,disp_data,SPqr = disptools.build_curve(SPs,eig_full[ifiles],q_full[ifiles])
    disp_inter = np.zeros((nintp*(len(SPs)-1),neigvals+1)) # interpolated dispersion data
    
    # Interpolate the data
    iqestart = 0
    intpx = np.linspace(qr[0], qr[-1], nintp*(len(SPs)-1))
    for ieig in range(neigvals):
        ydat = disp_data[:,3+ieig]
        f = CubicSpline(qr, ydat, bc_type='clamped')
        disp_inter[:, ieig+1] = f(intpx)

    disp_inter[:,0] = intpx

    disp_data_full.append(disp_data)
    disp_inter_full.append(disp_inter)

/home/wassimk/Documents/Molecular Dynamics Projects/Argon Phonon Properties/jupyter/md/post/disptools.py:41: ComplexWarning: Casting complex values to real discards the imaginary part
  eig[i,:],eigvec = np.linalg.eig(np.reshape(phi[i,:], (3,3)))


In [6]:
from bokeh.plotting import figure,output_file
from bokeh.io import show, output_notebook, push_notebook
from bokeh.models import ColumnDataSource, Panel, CustomJS, Circle, Line, Div, HoverTool
from bokeh.models.widgets import CheckboxGroup, Slider, RangeSlider
from bokeh.layouts import column, row
from bokeh.application import Application

output_file("argon_app.html")

# Create the figure
p = figure(title="Dispersion curves for FCC",
            y_range=(0, 2.25),
            x_range=(0, disp_inter[-1,0]+0.001),
            x_axis_label="q", y_axis_label="ω (THz)",
            plot_width=800)

neigvals = 3

raw_glyphs = []
inter_glyphs = []

draw = {"xraw":qr}
dint = {"xint":disp_inter[:,0]}
for ip in range(nfiles):
    for ie in range(neigvals):
        ei = "e"+str(ip)+"_"+str(ie)
        draw[ei] = disp_data_full[ip][:,ie+3]
        dint[ei] = disp_inter_full[ip][:,ie+1]

srcraw = ColumnDataSource(draw)
srcint = ColumnDataSource(dint)
for ip in range(nfiles):
    lalpha = 0
    for ie in range(neigvals):
        ei = "e"+str(ip)+"_"+str(ie)
        if ip==0: lalpha = 1
        raw_glyphs.append(Circle(x="xraw", y=ei, line_alpha=lalpha,fill_alpha=lalpha))
        inter_glyphs.append(Line(x="xint", y=ei, line_alpha=lalpha))
        p.add_glyph(srcraw,raw_glyphs[-1])
        p.add_glyph(srcint,inter_glyphs[-1])


# Styling
p.xaxis.ticker = SPqr
p.xaxis.major_label_overrides = dict(zip(SPqr,SPsNames))


# Widgets
EIGEN_LBLS = ["Eigen 1", "Eigen 2", "Eigen 3"]
eigen_chkbx_grp = CheckboxGroup(labels=EIGEN_LBLS, active=[i for i in range(neigvals)])
slider = Slider(start=1, end=nfiles, value=1, step=1, title="Pressure series")
pvals = [''.join(re.findall(regex, f)) for f in onlylogfiles ]
div = Div(text="Pressure = "+pvals[0]+" bar", name="pval")

slider.js_on_change("value", CustomJS(args=dict(glp=raw_glyphs,glq=inter_glyphs,neig=neigvals,lbl=div, vals=pvals, chkbx=eigen_chkbx_grp), code="""
    var i;
    var eigs = chkbx.active
    for (i=0; i<glp.length; i=i+1){
        if (i>=(this.value-1)*neig && i < this.value*neig){
            if( eigs.includes(i%neig) ){
                glp[i]["fill_alpha"] = 1;
                glp[i]["line_alpha"] = 1;
                glq[i]["line_alpha"] = 1;
            } else {
                glp[i]["fill_alpha"] = 0;
                glp[i]["line_alpha"] = 0;
                glq[i]["line_alpha"] = 0;
            }
            console.log(i,'YES')
        }else{
            glp[i]["fill_alpha"] = 0;
            glp[i]["line_alpha"] = 0;
            glq[i]["line_alpha"] = 0;
            console.log(i,'NO')
        }
    }
    lbl["text"] = "Pressure = " + vals[this.value-1] + " bar"
"""))


eigen_chkbx_grp.js_on_click(CustomJS(args=dict(glp=raw_glyphs,glq=inter_glyphs,neig=neigvals,sldr=slider), code="""
    var actv = this.active
    var i
    console.log("sldr ",sldr.value)
    for (i=(sldr.value-1)*neig; i<sldr.value*neig; i=i+1){
        console.log('i, i%neig res = ', i, i%neig, actv.includes(i%neig))
        if (actv.includes(i%neig)){
            glp[i]["fill_alpha"]=1;
            glp[i]["line_alpha"]=1;
            glq[i]["line_alpha"]=1;
        } else {
            glp[i]["fill_alpha"]=0;
            glp[i]["line_alpha"]=0;
            glq[i]["line_alpha"]=0;
        }
    }
"""))

hover = HoverTool(tooltips=[("ω (THz)", "$y")])
p.add_tools(hover)
layout = column(eigen_chkbx_grp,p,row(slider,div))
show(layout)

